### Choice simulation work

Sam Maurer, October 2018

This notebook contains benchmarks, feature development, and testing for ChoiceModels PR [#43](https://github.com/UDST/choicemodels/pull/43) and [#44](https://github.com/UDST/choicemodels/pull/44)

In [1]:
import numpy as np
import pandas as pd

In [2]:
import choicemodels
print(choicemodels.__version__)

0.2.dev4


### Benchmark df.apply vs matrix math for chooser-level random draws

There's no `numpy` function to perform simultaneous random draws from K distinct probability distributions, which we often need to do to simulate choices for K choosers.

Fletcher wrote an implementation using matrix math for `urbansim.urbanchoice.mnl`, which I refactored and generalized in `choicemodels.tools`. 

But I realized that in other places, like `urbansim.models.dcm`, we use `df.apply` for similar operations. This seems cleaner and more easily maintainable, and i'm curious how the performance compares. Maybe the matrix math implementation is only needed for things like GPU acceleration?

In [4]:
from choicemodels.tools import monte_carlo_choices

In [5]:
def generate_probs(n_obs, n_alts):
    n_obs = int(n_obs)
    n_alts = int(n_alts)
    
    d = {'oid': np.repeat(np.arange(n_obs), n_alts),
         'aid': np.tile(np.arange(n_alts), n_obs),
         'probs': np.random.random(n_obs * n_alts)}

    df = pd.DataFrame(d)
    df['probs'] = df.probs.div(df.groupby('oid').probs.transform('sum'))
    return df.set_index(['oid','aid']).probs

print(generate_probs(2,3))

oid  aid
0    0      0.290371
     1      0.164608
     2      0.545021
1    0      0.324821
     1      0.413295
     2      0.261884
Name: probs, dtype: float64


In [6]:
probs = generate_probs(1e4, 100)

#### 1. Matrix implementation from choicemodels

In [7]:
%%time
c = monte_carlo_choices(probs)
print(len(c))

10000
CPU times: user 21.8 ms, sys: 4.2 ms, total: 26 ms
Wall time: 24.3 ms


#### 2. df.apply

In [8]:
df = pd.DataFrame(probs).reset_index()

In [9]:
%%time
c = df.groupby('oid').apply(lambda x: np.random.choice(x.aid, p=x.probs))
print(len(c))

10000
CPU times: user 1.03 s, sys: 9.13 ms, total: 1.04 s
Wall time: 1.03 s


#### 3. Try keeping indexes to make it faster

In [10]:
def mkchoice(probs):
    return np.random.choice(probs.index.values, p=probs)

In [11]:
%%time
c = probs.groupby(level='oid', sort=False).apply(mkchoice)
print(len(c))

10000
CPU times: user 3.21 s, sys: 25.9 ms, total: 3.24 s
Wall time: 3.22 s


**df.apply is way slower!! At least 50x. We should use the matrix math implementation everywhere**

#### 4. What about np.random.choice with single probability distribution?

In [12]:
probs = np.random.random(100)
probs = probs/probs.sum()

In [13]:
%%time
c = np.random.choice(np.arange(100), size=int(1e4), replace=True, p=probs)

CPU times: user 1.46 ms, sys: 1.21 ms, total: 2.67 ms
Wall time: 1.56 ms


That's the way to go when there's a single probability distribution

### Benchmark choice simulation with capacity constraints

When the alternatives have capacity constraints, the fastest python approach seems to be simulate them iteratively: draw unconstrained choices, nullify the ones that exceed capacity constraints, and redraw. How much of a performance hit does this typically entail?

In [14]:
from choicemodels.tools import MergedChoiceTable, iterative_lottery_choices, monte_carlo_choices
from choicemodels import MultinomialLogit

In [15]:
n_obs = int(1e6)
n_alts = int(0.4e6)

d1 = {'oid': np.arange(n_obs), 
      'obsval': np.random.random(n_obs),
      'choice': np.random.choice(np.arange(n_alts), size=n_obs)}
obs = pd.DataFrame(d1).set_index('oid')

d2 = {'aid': np.arange(n_alts), 
      'altval': np.random.random(n_alts),
      'capacity': np.random.choice([1,3,5], n_alts)}
alts = pd.DataFrame(d2).set_index('aid')

mct = MergedChoiceTable(obs, alts, 'choice', sample_size=10)
m = MultinomialLogit(mct, model_expression='obsval + altval - 1')
fitted_model = m.fit()

In [16]:
def mct(obs, alts):
    return MergedChoiceTable(obs, alts, sample_size=10)

def probs(mct):
    return fitted_model.probabilities(mct)

#### Constrained, with about 20% excess capacity

In [17]:
%%time
c = iterative_lottery_choices(obs, alts, mct, probs, alt_capacity='capacity')

Iteration 1: 725218 of 1000000 valid choices
Iteration 2: 945331 of 1000000 valid choices
Iteration 3: 995382 of 1000000 valid choices
Iteration 4: 999954 of 1000000 valid choices
Iteration 5: 1000000 of 1000000 valid choices
CPU times: user 7.35 s, sys: 1.42 s, total: 8.77 s
Wall time: 7.9 s


#### Unconstrained

In [18]:
%%time
t = mct(obs, alts)

CPU times: user 3.24 s, sys: 703 ms, total: 3.94 s
Wall time: 3.94 s


In [19]:
%%time
p = probs(t)

CPU times: user 693 ms, sys: 132 ms, total: 825 ms
Wall time: 576 ms


In [20]:
%%time
c = monte_carlo_choices(p)

CPU times: user 300 ms, sys: 92.7 ms, total: 392 ms
Wall time: 391 ms
